# 多Agent协作

多个应用协作的案例

In [1]:
import shutup
shutup.please()

from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
from langchain.tools import Tool
from langchain.chains import LLMRequestsChain, LLMChain
from langchain.prompts import Prompt
from langchain.agents import initialize_agent

load_dotenv()
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)

接下来，定义几个业务tool:

In [2]:
# 通用处理逻辑
def generic_func(query):
    prompt = Prompt.from_template('回答问题：{query}')
    llm_chain = LLMChain(llm=llm, prompt=prompt)
    return llm_chain.run(query)

# 上网搜索逻辑
def search_func(query):
    prompt = Prompt.from_template('''
    请根据以下搜索结果，回答用户问题。
    搜索结果：
    {requests_result}
    问题：{query}
    ''')
    llm_chain = LLMChain(llm=llm, prompt=prompt)
    llm_request_chain = LLMRequestsChain(llm_chain=llm_chain)
    input = {
        'query': query,
        'url': 'https://www.google.com/search?q='+query.replace(' ', '+')
    }
    return llm_request_chain.run(input)

tools = [
    Tool(
        name='通用大模型',
        func=generic_func,
        description='利用大模型自身能力，回答问题。'
    ),
    Tool(
        name='搜索引擎',
        func=search_func,
        description='其他模型没有正确答案时，使用该工具。'
    ),
]

agent调用tool：

In [5]:
agent_chain = initialize_agent(tools=tools, llm=llm, verbose=True)

result = agent_chain.run('感冒吃什么药能好？') # 命中通用模型
result



> Entering new AgentExecutor chain...
I should use the general model to answer this question.
Action: 通用大模型
Action Input: 感冒吃什么药能好？
Observation: 感冒通常是由病毒引起的，因此抗生素在治疗感冒时并不起作用。一般来说，治疗感冒的药物包括退烧药（如对乙酰氨基酚）、止咳药、开塞露等药物。此外，及时休息、多喝水、吃清淡易消化的食物，可以帮助缓解感冒症状并加快康复。如果症状较为严重或持续时间较长，建议就医并按医生的建议进行治疗。
Thought:I now know the final answer
Final Answer: 感冒的药物治疗包括退烧药、止咳药、开塞露等，同时休息、补充水分、饮食清淡易消化有助于缓解症状。如果症状严重或持续，应就医并遵医嘱治疗。

> Finished chain.


'感冒的药物治疗包括退烧药、止咳药、开塞露等，同时休息、补充水分、饮食清淡易消化有助于缓解症状。如果症状严重或持续，应就医并遵医嘱治疗。'

In [7]:
result = agent_chain.run('双城之战第二季是什么时候开播的？') # 命中搜索引擎
result



> Entering new AgentExecutor chain...
I can use the search engine to find the answer to this question.
Action: 搜索引擎
Action Input: 双城之战第二季开播时间
Observation: 答案：《双城之战第二季》将于2024年11月9日在腾讯视频开播。
Thought:The show "双城之战第二季" will premiere on November 9, 2024, on Tencent Video.
Final Answer: 双城之战第二季将于2024年11月9日在腾讯视频开播。

> Finished chain.


'双城之战第二季将于2024年11月9日在腾讯视频开播。'

引入agent之后，就不需要人工来判断什么时候调用什么逻辑了，非常方便。

但是，刚刚的测试是无关联的，agent无上下文记忆，例如：

In [8]:
result = agent_chain.run('百日咳是一种什么病？')
result = agent_chain.run('吃什么药好得快？')
result



> Entering new AgentExecutor chain...
我应该首先尝试使用通用大模型回答这个问题。
Action: 通用大模型
Action Input: 百日咳是一种什么病？
Observation: 百日咳是一种由百日咳杆菌引起的传染病，主要症状包括剧烈的持续性咳嗽、呼吸困难、呕吐和发热等。百日咳主要通过飞沫传播，易在人群密集的地方传播，特别是婴幼儿和年幼儿童容易感染百日咳。预防百日咳的方法包括接种疫苗、保持好个人卫生等。如果患上百日咳，及时就医并接受治疗可以减轻症状和并发症的发生。
Thought:我现在知道百日咳是由百日咳杆菌引起的传染病，主要的症状包括剧烈的持续性咳嗽、呼吸困难、呕吐和发热等。
Final Answer: 百日咳是一种由百日咳杆菌引起的传染病，主要症状包括剧烈的持续性咳嗽、呼吸困难、呕吐和发热等。

> Finished chain.


> Entering new AgentExecutor chain...
I should use the search engine to find information on this topic.
Action: 搜索引擎
Action Input: 吃什么药好得快？
Observation: 根据搜索结果，如果你感冒了想快速好起来，可以考虑使用解热镇痛药，例如对乙酰氨基酚（泰诺）、布洛芬（布洛芬）等，可以有效降低体温和缓解头痛、身体不适等症状。但请注意，大多数药物都有一些副作用，建议咨询医生或药师的建议后再使用药物。
Thought:I now know the possible medications to take for a quick recovery.
Final Answer: 解热镇痛药，例如对乙酰氨基酚、布洛芬

> Finished chain.


'解热镇痛药，例如对乙酰氨基酚、布洛芬'

现在大模型没有承接到第一个问题的上下文，回答跑偏了，变成了感冒吃什么药好的快。